# Kmatrix input widget

Needs `ipywidgets` and `jupyterlab_widgets`
```console
$ pip install jupyterlab_widgets ipywidgets
```

Uncomment the code in following cell in your notebook, restart the kernel and reload the page to install it just now

In [ ]:
# !pip install jupyterlab_widgets ipywidgets

In [25]:
from __future__ import annotations

In [26]:
from collections import UserString
from typing import Callable

import ipywidgets as widgets
import yaml
from IPython.display import Markdown, display
from ipywidgets import GridspecLayout, Layout

## Define functions needed for the widget

In [34]:
class MarkdowStr(UserString):
    def __init__(self, markdown_str: str, *, syntax=None):
        self.data = markdown_str
        self.syntax = syntax

    def _repr_markdown_(self):
        if self.syntax is not None:
            return f"```{self.syntax}\n{self.data}\n```"
        else:
            return self.data
        
    def __str__(self):
        return self._repr_markdown_()

In [28]:
def create_input_grid(compartments: list[str]):
    size = len(compartments)
    grid = GridspecLayout(size, size, width=f"{(size*1.1)**2}em")
    for i in range(size):
        for j in range(size):
            grid[i, j] = widgets.Text(layout=Layout(height="4em", width="4em"))
    return grid

In [29]:
compartments = [f"s{i}" for i in range(1, 5)]
grid = create_input_grid(compartments)
grid

GridspecLayout(children=(Text(value='', layout=Layout(grid_area='widget001', height='4em', width='4em')), Text…

In [30]:
def grid_to_parameter(grid: GridspecLayout, *, parameter_name: str = "kinetic"):
    parameters = set()
    for child in grid.children:
        if not isinstance(child, widgets.Widget):
            continue
        value = child.value
        if value == "":
            continue
        parameters.add(f'["{value}", 1e-2, {{vary: true}}]')

    values = {parameter_name: sorted(list(parameters))}
    return MarkdowStr(
        yaml.dump(values).replace("'", "").replace("- ", "  - "), syntax="yaml"
    )


grid_to_parameter(grid)

```yaml
kinetic: []

```

In [31]:
def grid_to_matrix(
    grid: GridspecLayout,
    compartments: list[str],
    *,
    matrix_name: str = "km1",
    parameter_name: str = "kinetic",
):
    matrix_entries = {}
    size = len(compartments)
    for i in range(size):
        for j in range(size):
            child = grid[i, j]
            if not isinstance(child, widgets.Widget):
                continue
            value = child.value
            if value != "":
                key = f"({compartments[i]}, {compartments[j]})"
                matrix_entries[key] = f"{parameter_name}.{value}"
    values = {"k_matrix": {matrix_name: {"matrix": matrix_entries}}}
    return MarkdowStr(yaml.dump(values), syntax="yaml")


grid_to_matrix(grid, compartments, matrix_name="km3")

```yaml
k_matrix:
  km3:
    matrix: {}

```

In [32]:
def bind_callbacks(grid: GridspecLayout, callback: Callable[[...], ...]):
    for child in grid.children:
        if isinstance(child, widgets.Widget):
            child.observe(callback)

## Final widget function 
This function allows quickly converting a legacy K-Matrix to the new pyglotaran yaml style.

In [39]:
def matrix_helper(
    compartments: list[str],
    *,
    matrix_name="km1",
    parameter_name="kinetic",
):
    grid = create_input_grid(compartments)
    out = widgets.Output()

    def display_yml(*args, **kwargs):
        out.clear_output(wait=True)
        with out:
            matrix_str = grid_to_matrix(
                grid=grid,
                compartments=compartments,
                matrix_name="km1",
                parameter_name=parameter_name,
            )

            parameter_str = grid_to_parameter(
                grid=grid, parameter_name=parameter_name
            )

            model_str = f"Add the following to your `model.yml`: \n{matrix_str}\n"
            parameter_str = (
                f"Add the following to your `parameter.yml`: \n{parameter_str}"
            )

            display_obj = Markdown(f"{model_str} <br>{parameter_str}")
            display(display_obj)
            

    bind_callbacks(grid, display_yml)
    return widgets.VBox([grid, out])


matrix_helper([*compartments, "s5"])

## Format notebook

In [ ]:
# !nbqa black utils.ipynb --nbqa-mutate
# !nbqa isort utils.ipynb --nbqa-mutate